In [ ]:
import folium
import pandas as pd

In [ ]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [ ]:
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv")
spacex_df.head(5)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Mark all launch sites on a map

In [ ]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_site_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_site_df = launch_site_df[['Launch Site', 'Lat', 'Long']]
launch_site_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [ ]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

In [ ]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('Nasa Johnson Space'))

marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )

site_map.add_child(circle)
site_map.add_child(marker)

In [ ]:
for i in range(len(launch_site_df)):
  coordinate = [launch_site_df.iloc[i]['Lat'], launch_site_df.iloc[i]['Long']]
  name = launch_site_df.iloc[i]['Launch Site']
  circle = circle = folium.Circle(coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(f'{name}'))

  marker = folium.map.Marker(
    coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % f'{name}',
        )
    )

  site_map.add_child(circle)
  site_map.add_child(marker)

Mark the success/failed launches for each site on the map

In [ ]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [ ]:
marker_cluster = MarkerCluster()

In [ ]:
spacex_df['marker_color'] = spacex_df['class'].map(lambda x: 'green' if x==1 else 'red')

spacex_df.tail(10)

<ipython-input-122-0a3ad48119eb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spacex_df['marker_color'] = spacex_df['class'].map(lambda x: 'green' if x==1 else 'red')


,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [ ]:
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
  coordinate = [record['Lat'], record['Long']]
  marker_color = record['marker_color']
  marker = folium.map.Marker(coordinate, icon=folium.Icon(color=marker_color))
  marker_cluster.add_child(marker)

site_map

Calculate the distances between a launch site to its proximities

In [ ]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
coastline_lat = 28.5623
coastline_lon = -80.570
distance_coastline = calculate_distance(launch_site_df.iloc[0]['Lat'], launch_site_df.iloc[0]['Long'], coastline_lat, coastline_lon)
distance_coastline

0.7186759894147521

In [ ]:
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon = DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),)
)

site_map.add_child(distance_marker)

In [ ]:
coordinates = [[launch_site_df.iloc[0]['Lat'], launch_site_df.iloc[0]['Long']], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [ ]:
closest_highway = 28.56152, -80.57731
closest_railroad = 28.57125, -80.5854
closest_city = 28.60502, -80.83191

In [ ]:
distance_highway = calculate_distance(launch_site_df.iloc[0]['Lat'], launch_site_df.iloc[0]['Long'], closest_highway[0], closest_highway[1])
print('distance_highway =',distance_highway, ' km')
distance_railroad = calculate_distance(launch_site_df.iloc[0]['Lat'], launch_site_df.iloc[0]['Long'], closest_railroad[0], closest_railroad[1])
print('distance_railroad =',distance_railroad, ' km')
distance_city = calculate_distance(launch_site_df.iloc[0]['Lat'], launch_site_df.iloc[0]['Long'], closest_city[0], closest_city[1])
print('distance_city =',distance_city, ' km')

distance_highway = 0.08709683983247721  km
distance_railroad = 1.2680769213842178  km
distance_city = 25.312975413295238  km


In [ ]:
distance_highway_marker = folium.Marker(
    closest_highway,
    icon = DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),)
)

coordinates = [[launch_site_df.iloc[0]['Lat'], launch_site_df.iloc[0]['Long']], [closest_highway[0], closest_highway[1]]]
lines_highway = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

site_map.add_child(lines_highway)
site_map.add_child(distance_highway_marker)

In [ ]:
distance_railroad_marker = folium.Marker(
    closest_railroad,
    icon = DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railroad),)
)

coordinates = [[launch_site_df.iloc[0]['Lat'], launch_site_df.iloc[0]['Long']], [closest_railroad[0], closest_railroad[1]]]
lines_railroad = folium.PolyLine(locations=coordinates, weight=1)

site_map.add_child(lines_railroad)
site_map.add_child(distance_railroad_marker)

In [ ]:
distance_city_marker = folium.Marker(
    closest_city,
    icon = DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),)
)

coordinates = [[launch_site_df.iloc[0]['Lat'], launch_site_df.iloc[0]['Long']], [closest_city[0], closest_city[1]]]
lines_city = folium.PolyLine(locations=coordinates, weight=1)

site_map.add_child(lines_city)
site_map.add_child(distance_city_marker)